<a href="https://colab.research.google.com/github/DjengoFang/Financial-Engineering-Ruppert-Matteson-/blob/main/Lab6.5.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install bootstrap_stat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 12.5 MB/s eta 0:00:00


In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy.stats as stats
import statsmodels.api as sm
import pylab as py
import seaborn as sns
import random
from tqdm import tqdm
from bootstrap_stat import bootstrap_stat as bp
from sklearn.utils import resample

In [9]:
niter = 500
nboot = 500
n = 50
nu = 10
trueKurtosis = 3 + 6/(nu-4)
correct = np.empty(shape=(5,niter))
width = np.empty(shape=(5,niter))
error = np.empty(shape=(niter))

In [4]:
def kurtosis(df):
  return np.mean(((df-np.mean(df))/np.std(df))**4)

In [8]:
def check(low, high):
  if low < trueKurtosis and high > trueKurtosis:
    return 1
  return 0

In [14]:
for i in tqdm(range(niter)):
  y = stats.t.rvs(df=nu, size=n)
  dist = bp.EmpiricalDistribution(y)
  boot = []
  for j in range(nboot):
    sample = resample(y)
    boot.append(kurtosis(sample))

  # Bootstrap-t Intervals
  theta_hat = kurtosis(y)
  c1_low, c1_high = bp.t_interval(dist, kurtosis, theta_hat, alpha=0.025)
  correct[0][i] = check(c1_low, c1_high)
  width[0][i] = c1_high - c1_low

  # BCa Confidence Intervals
  c2_low, c2_high = bp.bcanon_interval(dist, kurtosis, y, alpha=0.025)
  correct[1][i] = check(c2_low, c2_high)
  width[1][i] = c2_high - c2_low
  
  # Normal Approximation Intervals
  c3_low = theta_hat - 1.96*np.std(boot)
  c3_high = theta_hat + 1.96*np.std(boot)
  correct[2][i] = check(c3_low, c3_high)
  width[2][i] = c3_high - c3_low

  # Percentile Confidence Intervals
  c4_low, c4_high = np.quantile(boot, [0.025, 0.975])
  correct[3][i] = check(c4_low, c4_high)
  width[3][i] = c4_high - c4_low

  # Basic Bootstrap Intervals
  c5_low = 2*theta_hat - np.quantile(boot, 0.975)
  c5_high = 2*theta_hat - np.quantile(boot, 0.025)
  correct[4][i] = check(c5_low, c5_high)
  width[4][i] = c5_high - c5_low

  error[i] = np.mean(boot) - theta_hat

100%|██████████| 500/500 [23:51<00:00,  2.86s/it]


In [15]:
print(np.mean(correct, axis=1))
print(np.mean(width, axis=1))
print(np.mean(error))
print(np.mean(error**2))

[0.71  0.748 0.618 0.65  0.532]
[6.7362648  3.51282801 2.86906488 2.72721726 2.72721726]
-0.2019978965798473
0.1898881829880655
